In [1]:
# Import necessary libraries
import networkx as nx 
from networkx.exception import NetworkXError

In [2]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, 
             nstart=None, weight='weight', dangling=None): 

    if len(G) == 0: 
        return {} 
    if not G.is_directed(): 
        D = G.to_directed() 
    else: 
        D = G 

    # Create a copy in (right) stochastic form 
    W = nx.stochastic_graph(D, weight=weight) 
    N = W.number_of_nodes() 

    # Choose fixed starting vector if not given 
    if nstart is None: 
        x = dict.fromkeys(W, 1.0 / N) 
    else: 
        # Normalized nstart vector 
        s = float(sum(nstart.values())) 
        x = dict((k, v / s) for k, v in nstart.items()) 

    if personalization is None: 
        # Assign uniform personalization vector if not given 
        p = dict.fromkeys(W, 1.0 / N) 
    else: 
        missing = set(G) - set(personalization) 
        if missing: 
            raise NetworkXError('Personalization dictionary must have a value for every node. '
                                ' Missing nodes %s' % missing) 
        s = float(sum(personalization.values())) 
        p = dict((k, v / s) for k, v in personalization.items()) 

    if dangling is None: 
        # Use personalization vector if dangling vector not specified 
        dangling_weights = p 
    else: 
        missing = set(G) - set(dangling) 
        if missing: 
            raise NetworkXError('Dangling node dictionary must have a value for every node. '
                                'Missing nodes %s' % missing) 
        s = float(sum(dangling.values())) 
        dangling_weights = dict((k, v/s) for k, v in dangling.items()) 
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0] 

    # power iteration: make up to max_iter iterations 
    for _ in range(max_iter): 
        xlast = x 
        x = dict.fromkeys(xlast.keys(), 0) 
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes) 
        for n in x: 
            # this matrix multiply looks odd because it is 
            # doing a left multiply x^T=xlast^T*W 
            for nbr in W[n]: 
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight] 
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n] 
        # check convergence, l1 norm 
        err = sum([abs(x[n] - xlast[n]) for n in x]) 
        if err < N*tol: 
            return x 

    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter) 

In [3]:
G=nx.barabasi_albert_graph(60,41) 
pr=nx.pagerank(G,0.4) 
pr 

{0: 0.02798909453932378,
 1: 0.013176226490010075,
 2: 0.012543974790032827,
 3: 0.0125633665059297,
 4: 0.01337673141185962,
 5: 0.013381471466771367,
 6: 0.013575814762609998,
 7: 0.012971721129320195,
 8: 0.01274922256130476,
 9: 0.01277666584092255,
 10: 0.012364904708174442,
 11: 0.013155623246336168,
 12: 0.012774526201284197,
 13: 0.012950897461788763,
 14: 0.012572623732894538,
 15: 0.013182689995143985,
 16: 0.01275720283292269,
 17: 0.01297425762142516,
 18: 0.013371088978062592,
 19: 0.01357862928365683,
 20: 0.01275773107965887,
 21: 0.013369487491192794,
 22: 0.012569610577560392,
 23: 0.012547807194234274,
 24: 0.013168284714900124,
 25: 0.01337798260548712,
 26: 0.012567895279522519,
 27: 0.012974370639014111,
 28: 0.013161204286829794,
 29: 0.013152894414681171,
 30: 0.012975757065503683,
 31: 0.012383730022267003,
 32: 0.012363023679373426,
 33: 0.012968818651035535,
 34: 0.012989013440387946,
 35: 0.01274362176584864,
 36: 0.013175578571822917,
 37: 0.0125569076395103